In [33]:
import pandas as pd
import numpy as np

In [31]:
from sklearn.feature_extraction import DictVectorizer
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error

## Q1. Downloading the data

We'll use [the same NYC taxi dataset](https://www1.nyc.gov/site/tlc/about/tlc-trip-record-data.page),
but instead of "**Green** Taxi Trip Records", we'll use "**Yellow** Taxi Trip Records".

Download the data for January and February 2023.

Read the data for January. How many columns are there?

* 16
* 17
* 18
* 19

In [3]:
!wget -P ./data https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2023-01.parquet

--2024-05-20 03:15:42--  https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2023-01.parquet
Resolving d37ci6vzurychx.cloudfront.net (d37ci6vzurychx.cloudfront.net)... 13.32.192.2, 13.32.192.116, 13.32.192.124, ...
Connecting to d37ci6vzurychx.cloudfront.net (d37ci6vzurychx.cloudfront.net)|13.32.192.2|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 47673370 (45M) [application/x-www-form-urlencoded]
Saving to: ‘./data/yellow_tripdata_2023-01.parquet.1’

yellow_tripdata_202 100%[===================>]  45.46M  73.2MB/s    in 0.6s    

2024-05-20 03:15:42 (73.2 MB/s) - ‘./data/yellow_tripdata_2023-01.parquet.1’ saved [47673370/47673370]



In [4]:
!wget -P ./data https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2023-02.parquet

--2024-05-20 03:15:43--  https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2023-02.parquet
Resolving d37ci6vzurychx.cloudfront.net (d37ci6vzurychx.cloudfront.net)... 13.32.192.190, 13.32.192.124, 13.32.192.116, ...
Connecting to d37ci6vzurychx.cloudfront.net (d37ci6vzurychx.cloudfront.net)|13.32.192.190|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 47748012 (46M) [application/x-www-form-urlencoded]
Saving to: ‘./data/yellow_tripdata_2023-02.parquet.1’

yellow_tripdata_202 100%[===================>]  45.54M   124MB/s    in 0.4s    

2024-05-20 03:15:43 (124 MB/s) - ‘./data/yellow_tripdata_2023-02.parquet.1’ saved [47748012/47748012]



In [6]:
!ls -lh data/

total 94M
-rw-rw-r-- 1 ubuntu ubuntu 1.3M Jun 30  2022 green_tripdata_2021-01.parquet
-rw-rw-r-- 1 ubuntu ubuntu 1.1M Jun 30  2022 green_tripdata_2021-02.parquet
-rw-rw-r-- 1 ubuntu ubuntu  46M Mar 20  2023 yellow_tripdata_2023-01.parquet
-rw-rw-r-- 1 ubuntu ubuntu  46M May  1  2023 yellow_tripdata_2023-02.parquet


In [5]:
df=pd.read_parquet("./data/yellow_tripdata_2023-01.parquet")

In [7]:
print(f"Total Columns: {df.shape[1]}")

Total Columns: 19


## Q2. Computing duration

Now let's compute the `duration` variable. It should contain the duration of a ride in minutes. 

What's the standard deviation of the trips duration in January?

* 32.59
* 42.59
* 52.59
* 62.59

In [8]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3066766 entries, 0 to 3066765
Data columns (total 19 columns):
 #   Column                 Dtype         
---  ------                 -----         
 0   VendorID               int64         
 1   tpep_pickup_datetime   datetime64[us]
 2   tpep_dropoff_datetime  datetime64[us]
 3   passenger_count        float64       
 4   trip_distance          float64       
 5   RatecodeID             float64       
 6   store_and_fwd_flag     object        
 7   PULocationID           int64         
 8   DOLocationID           int64         
 9   payment_type           int64         
 10  fare_amount            float64       
 11  extra                  float64       
 12  mta_tax                float64       
 13  tip_amount             float64       
 14  tolls_amount           float64       
 15  improvement_surcharge  float64       
 16  total_amount           float64       
 17  congestion_surcharge   float64       
 18  airport_fee           

In [9]:
df['duration']=df.tpep_dropoff_datetime - df.tpep_pickup_datetime
df['duration']=df['duration'].apply(lambda td:td.total_seconds()/60)

In [10]:
df['duration'].std()

42.594351241920904

## Q3. Dropping outliers

Next, we need to check the distribution of the `duration` variable. There are some outliers. Let's remove them and keep only the records where the duration was between 1 and 60 minutes (inclusive).

What fraction of the records left after you dropped the outliers?

* 90%
* 92%
* 95%
* 98%

In [14]:
new_df=df[(df.duration>=1) & (df.duration<=60)]

In [15]:
fraction_percentage=len(new_df)/len(df)*100

In [16]:

print(f"Fraction of records left: {fraction_percentage:.2f}%")

Fraction of records left: 98.12%


## Q4. One-hot encoding

Let's apply one-hot encoding to the pickup and dropoff location IDs. We'll use only these two features for our model. 

* Turn the dataframe into a list of dictionaries (remember to re-cast the ids to strings - otherwise it will 
  label encode them)
* Fit a dictionary vectorizer 
* Get a feature matrix from it

What's the dimensionality of this matrix (number of columns)?

* 2
* 155
* 345
* 515
* 715

In [17]:
def read_dataframe(filename):
    
    df=pd.read_parquet(filename)

    df['duration']=df.tpep_dropoff_datetime - df.tpep_pickup_datetime
    df.duration=df.duration.apply(lambda td:td.total_seconds()/60)
    
    df=df[(df.duration>=1) & (df.duration<=60)]
    
    categorical=['PULocationID','DOLocationID']
    
    df[categorical]=df[categorical].astype(str)
    return df

In [18]:
df_train=read_dataframe("./data/yellow_tripdata_2023-01.parquet")

In [19]:
len(df_train)

3009173

In [20]:
categorical = ['PULocationID', 'DOLocationID']

In [21]:
dv=DictVectorizer()
train_dicts=df_train[categorical].to_dict(orient="records")
X_train=dv.fit_transform(train_dicts)






In [22]:
X_train.shape

(3009173, 515)

## Q5. Training a model

Now let's use the feature matrix from the previous step to train a model. 

* Train a plain linear regression model with default parameters 
* Calculate the RMSE of the model on the training data

What's the RMSE on train?

* 3.64
* 7.64
* 11.64
* 16.64

In [23]:
target='duration'
y_train=df_train[target].values



In [27]:
lr=LinearRegression()
lr.fit(X_train,y_train)






LinearRegression()

In [28]:
# Making predictions on the training data
y_pred = lr.predict(X_train)

In [35]:
mean_squared_error(y_train,y_pred,squared=False)

7.649261927665777

## Q6. Evaluating the model

Now let's apply this model to the validation dataset (February 2023). 

What's the RMSE on validation?

* 3.81
* 7.81
* 11.81
* 16.81

In [36]:
df_val=read_dataframe("./data/yellow_tripdata_2023-02.parquet")

In [37]:
len(df_val)

2855951

In [38]:
val_dicts=df_val[categorical].to_dict(orient="records")
X_val=dv.transform(val_dicts)

In [39]:
y_val=df_val[target].values

In [40]:
y_pred=lr.predict(X_val)

mean_squared_error(y_val,y_pred,squared=False)

7.811817548344513